# Visualização de Dados de Saúde dos Municípios Brasileiros


Ao final desse Notebook devemos visualizar de forma simples e didática os seguintes dados:
 
- [ ] População dos municípios brasileiros
- [ ] Quantidade de hospitais públicos e não públicos por município brasileiro
- [ ] Quantidade de enfermeiros do SUS ou não por município brasileiro
- [ ] Quantidade de médicos do SUS ou não por município brasileiro
- [ ] Quantidade de leitos de internação do SUS ou não por município brasileiro
- [ ] Quantidade de leitos de UTI do SUS ou não por município brasileiro
- [ ] Quantidade de leitos de UTI para COVID-19 do SUS ou não por município brasileiro
- [ ] Enfermeiros a cada 10k habitantes por município brasileiro
- [ ] Médicos a cada 10k habitantes por município brasileiro
- [ ] Hospitais a cada 10k habitantes por município brasileiro
- [ ] Leitos (internação) a cada 10k habitantes por município brasileiro
- [ ] Leitos UTI a cada 10k habitantes por município brasileiro
- [ ] Leitos UTI COVID-19 a cada 10k habitantes por município brasileiro

In [1]:
import pandas as pd
import ipywidgets
import json
import plotly
import plotly.graph_objects as go

In [2]:
# Leituras do df contendo os dados por município
df = pd.read_csv('..//data//processed//Brasil - Municípios.csv', encoding='utf-8')

states = list(df["UF"].unique())
dict_of_df = {}

for uf in states:
    key_name = str(uf)
    dict_of_df[key_name] = df[df['UF']== uf]

Inicialmente separamos o data frame _*'Brasil - Municípios'*_ em **27** data frames diferentes, um com os dados de cada Estado brasileiro.

Após essa separação, adicionamos todos esses dfs em um dicionário, cujas *keys* serão os códigos dos estados.

Ou seja, para acessar o dataframe contendo os dados dos municípios do estado de São Paulo, o código seria:

> dict_of_df['SP']

In [3]:
dict_of_df['SP']

,Unnamed: 0,Código Municipal,Município,População,UF,Enfermeiros SUS,Enfermeiros não-SUS,Enfermeiros - Total,% de enfermeiros SUS,Enfermeiros a cada 10k habitantes,...,Leitos UTI SUS,Leitos UTI não-SUS,Leitos UTI - Total,% de leitos UTI SUS,Leitos UTI a cada 10k habitantes,Leitos UTI COVID-19 SUS,Leitos UTI COVID-19 não-SUS,Leitos UTI COVID-19 - Total,% de leitos UTI COVID-19 SUS,Leitos UTI COVID-19 a cada 10k habitantes
3267,3267,350010,Adamantina,35068,SP,80,3,83,96.39,23.67,...,4,11,15,26.67,4.28,0,5,5,0.0,1.43
3268,3268,350020,Adolfo,3562,SP,6,0,6,100.00,16.84,...,0,0,0,0.00,0.00,0,0,0,0.0,0.00
3269,3269,350030,Aguaí,36305,SP,23,0,23,100.00,6.34,...,0,0,0,0.00,0.00,0,0,0,0.0,0.00
3270,3270,350040,Águas da Prata,8180,SP,5,1,6,83.33,7.33,...,0,0,0,0.00,0.00,0,0,0,0.0,0.00
3271,3271,350050,Águas de Lindóia,18705,SP,19,1,20,95.00,10.69,...,0,0,0,0.00,0.00,0,0,0,0.0,0.00
3272,3272,350055,Águas de Santa Bárbara,6075,SP,11,0,11,100.00,18.11,...,0,0,0,0.00,0.00,0,0,0,0.0,0.00
3273,3273,350060,Águas de São Pedro,3451,SP,6,0,6,100.00,17.39,...,0,0,0,0.00,0.00,0,0,0,0.0,0.00
3274,3274,350070,Agudos,37214,SP,48,0,48,100.00,12.90,...,0,0,0,0.00,0.00,0,0,0,0.0,0.00
3275,3275,350075,Alambari,6025,SP,7,0,7,100.00,11.62,...,0,0,0,0.00,0.00,0,0,0,0.0,0.00
3276,3276,350080,Alfredo Marcondes,4166,SP,2,0,2,100.00,4.80,...,0,0,0,0.00,0.00,0,0,0,0.0,0.00


In [4]:
# filtros a serem utilizados no mapa
filtros = list(df.loc[:,'População':].columns)

# densidades a serem utilizadas no mapa
densidades = [s for s in filtros if "habitantes" in s]

Além disso, criamos uma lista contendo os nomes das colunas que farão parte dos filtros do gráfico.

Utilizamos as densidades de hospitais, enfermeiros, médicos e leitos (UTI, internação e COVID) para
definir as "temperaturas" do mapa (cores).

A lista abaixo contém as opções para o usuário selecionar o gráfico desejado:

In [5]:
densidades

['Enfermeiros a cada 10k habitantes',
 'Médicos a cada 10k habitantes',
 'Hospitais a cada 100k habitantes',
 'Leitos (internação) a cada 10k habitantes',
 'Leitos UTI a cada 10k habitantes',
 'Leitos UTI COVID-19 a cada 10k habitantes']

Além de poder selecionar a informação desejada, o usuário também poderá selecionar o estado a ser analizado por meio de um botão de seleção.

In [6]:
# Nome das colunas com dados de enfermeiros
col_enfermeiros = [s for s in filtros if ("nfermeiros" in s or "População" in s)]

# Nome das colunas com dados de médicos
col_medicos = [s for s in filtros if ("dicos" in s or "População" in s)]

# Nome das colunas com dados de hospitais
col_hospitais = [s for s in filtros if ("ospitais" in s or "População" in s)]

# Nome das colunas com dados de leitos (internação)
col_leitos_int = [s for s in filtros if ("eitos (internação)" in s or "População" in s)]

# Nome das colunas com dados de leitos UTI
col_leitos_UTI = [s for s in filtros if (("eitos UTI" in s and "COVID" not in s) or "População" in s)]

# Nome das colunas com dados de leitos UTI COVID
col_leitos_UTI_COVID = [s for s in filtros if ("eitos UTI COVID" in s or "População" in s)]

# Dicionário contendo os 'tipos' de dados e suas respectivas colunas associadas
types = {'col_enfermeiros' : col_enfermeiros,
         'col_medicos' : col_medicos,
         'col_hospitais' : col_hospitais,
         'col_leitos_int' : col_leitos_int,
         'col_leitos_UTI' : col_leitos_UTI,
         'col_leitos_UTI_COVID' : col_leitos_UTI_COVID}

Também plotamos um _Tooltip_ ou _Hover_ para uma visualização mais detalhada dos dados por município. 

As informações contidas nesse Tooltip variam conforme o gráfico escolhido no filtro comentado anteriormente.

No dicionário baixo podemos consultar as informações presentes nos Tooltips de cada um dos gráficos a serem escolhidos.

In [7]:
types

{'col_enfermeiros': ['População',
  'Enfermeiros SUS',
  'Enfermeiros não-SUS',
  'Enfermeiros - Total',
  '% de enfermeiros SUS',
  'Enfermeiros a cada 10k habitantes'],
 'col_medicos': ['População',
  'Médicos SUS',
  'Médicos não-SUS',
  'Médicos - Total',
  '% de médicos SUS',
  'Médicos a cada 10k habitantes'],
 'col_hospitais': ['População',
  'Hospitais públicos',
  'Hospitais não-públicos',
  'Hospitais - Total',
  '% de hospitais públicos',
  'Hospitais a cada 100k habitantes'],
 'col_leitos_int': ['População',
  'Leitos (internação) SUS',
  'Leitos (internação) não-SUS',
  'Leitos (internação) - Total',
  '% de leitos (internação) SUS',
  'Leitos (internação) a cada 10k habitantes'],
 'col_leitos_UTI': ['População',
  'Leitos UTI SUS',
  'Leitos UTI não-SUS',
  'Leitos UTI - Total',
  '% de leitos UTI SUS',
  'Leitos UTI a cada 10k habitantes'],
 'col_leitos_UTI_COVID': ['População',
  'Leitos UTI COVID-19 SUS',
  'Leitos UTI COVID-19 não-SUS',
  'Leitos UTI COVID-19 - Total'

In [8]:
# Adiciona uma coluna no dict_of_df[<uf>] com o texto que estará no tooltip do gráfico
#for l in states:
#    for j in types:
#        dict_of_df['{}'.format(l)][j] = dict_of_df['{}'.format(l)]['Município'] + '<br><br>'
#        for i in range(6):
#            for k in range(len(dict_of_df['{}'.format(l)])):
#                dict_of_df['{}'.format(l)].loc[dict_of_df['{}'.format(l)].index[k],j] = dict_of_df['{}'.format(l)].loc[dict_of_df['{}'.format(l)].index[k],j] +\
#                str(types[j][i]) + ':  ' + str(dict_of_df['{}'.format(l)].loc[dict_of_df['{}'.format(l)].index[k],types[j][i]]) + '<br>'

In [9]:
# Como o processamento dessa nova coluna com o texto do tooltip ficou muito demorado, 
#salvamos os novos dfs
#for i in states:
#    dict_of_df['{}'.format(i)].to_csv('..\\data\\processed\\dados_municipios_por_estado\\{}\\{}.csv'.format(i, i), index = False, header=True, encoding='latin')

Para os textos dos **Tooltips**, adicionamos uma coluna ao nossos *dfs* contendo as informações a serem disponibilizadas.

Como levou um certo tempo para o processamento dessas informações, salvamos o df após todas as mudanças realizadas no formato .csv.

O arquivo pode ser encontrado no seguinte caminho:

- '..\\data\\processed\\dados_municipios_por_estado\\< UF >\\< UF >.csv'

In [10]:
dict_of_dff = {}
for uf in states:
    key_name = str(uf)
    dict_of_dff[key_name] = pd.read_csv("..\\data\\processed\\dados_municipios_por_estado\\{}\\{}.csv".format(uf, uf), encoding='latin-1')

Para plotarmos os contornos dos municípios do Brasil, precisamos de arquivos .json com o mapeamento de todos as latitudes e longitudes necessárias para desenhar esses contornos.

Esses arquivos foram encontrados em:

- [contorno dos municípios por estado brasileiro](https://github.com/tbrugz/geodata-br)

In [11]:
# Le os jsons que contêm todo o 'desenho' dos municípios brasileiros
dict_of_jsons = {}
for uf in states:
    key_name = str(uf)
    dict_of_jsons[key_name] = json.load(open("..\\data\\processed\\dados_municipios_por_estado\\{}\\br_{}.json".format(uf, uf), encoding='utf-8'))

Foi feita uma comparação entre os nomes dos municípios existentes no *df* e nos arquivos *.json* dos contornos.
Devido a algumas divergências, fizemos uma padronização dos nomes dos municípios.

In [12]:
lista = []
for j in states:
    k = str(j)
    for i in range(len(dict_of_jsons[k]['features'])):
        lista.append(dict_of_jsons[k]['features'][i]['properties']['name'])
    lissst = list(dict_of_dff[k]['Município'])
    print('\nMUNICIPIOS FALTANTES DE {}\n'.format(j))
    for i in lissst:
        if(i not in lista):
            print("n tem no json: {}".format(i))
    for i in lista:
        if(i not in lissst):
            print("n tem no df: {}".format(i))
    lista = []


MUNICIPIOS FALTANTES DE RO


MUNICIPIOS FALTANTES DE AC


MUNICIPIOS FALTANTES DE AM


MUNICIPIOS FALTANTES DE RR


MUNICIPIOS FALTANTES DE PA

n tem no json: Eldorado do Carajás
n tem no json: Mojuí dos Campos
n tem no json: Santa Izabel do Pará
n tem no df: Eldorado dos Carajás
n tem no df: Santa Isabel do Pará

MUNICIPIOS FALTANTES DE AP


MUNICIPIOS FALTANTES DE TO

n tem no json: São Valério
n tem no df: São Valério da Natividade

MUNICIPIOS FALTANTES DE MA


MUNICIPIOS FALTANTES DE PI

n tem no json: Nazária

MUNICIPIOS FALTANTES DE CE


MUNICIPIOS FALTANTES DE RN

n tem no json: Serra Caiada
n tem no df: Presidente Juscelino

MUNICIPIOS FALTANTES DE PB

n tem no json: Joca Claudino
n tem no json: São Vicente do Seridó
n tem no json: Tacima
n tem no df: Santarém
n tem no df: Seridó
n tem no df: Campo de Santana

MUNICIPIOS FALTANTES DE PE

n tem no json: Belém do São Francisco
n tem no json: Iguaracy
n tem no json: Lagoa de Itaenga
n tem no df: Belém de São Francisco
n tem no df

A partir das divergências encontradas entre os nomes dos municípios no arquivo .json e no banco de dados utilizado, descobrimos algumas curiosidades fora do tema a ser analisado:

**CURIOSIDADES**:

- *Santarém* é o antigo nome do município *Joca Claudino*, na Paraíba, este nome foi uma homenagem feita em 2010, a um importante comerciante para a história local.

- *São Valério da Natividade* é o antigo nome do município *São Valério*.

- *Nazária*, no Piauí, foi emancipada em 1993, virou município em 1995, mas seu estatuto só foi definido em 2005 devido a problemas jurídicos. Além disso, seu primeiro prefeito foi eleito apenas em 2008.

- *Serra da Caiada*, no Rio Grande do Norte, virou município em 1953. Em 1963, o município mudou o nome para Presidente Juscelino, mas a população não assimilou essa mudança. Em 1964, seu nome passou a ser Presidente Kubitschek. Em 2012, fizeram um plebiscito e, com 98,53% dos votos, o município voltou a ser chamado de Serra da Caiada. ~~UFA~~

- Em 1996, um Frei mudou o nome de *Tacima* para Campo de Santana, mas, em 2009 voltou a ser chamada Tacima.

In [13]:
for i in range(len(dict_of_jsons['PA']['features'])):
    if(dict_of_jsons['PA']['features'][i]['properties']['name'] == 'Eldorado dos Carajás'):
        dict_of_jsons['PA']['features'][i]['properties']['name'] = 'Eldorado do Carajás'
    if(dict_of_jsons['PA']['features'][i]['properties']['name'] == 'Santa Isabel do Pará'):
        dict_of_jsons['PA']['features'][i]['properties']['name'] = 'Santa Izabel do Pará'

for i in range(len(dict_of_jsons['TO']['features'])):
    if(dict_of_jsons['TO']['features'][i]['properties']['name'] == 'São Valério da Natividade'):
        dict_of_jsons['TO']['features'][i]['properties']['name'] = 'São Valério'

for i in range(len(dict_of_jsons['RN']['features'])):
    if(dict_of_jsons['RN']['features'][i]['properties']['name'] == 'Presidente Juscelino'):
        dict_of_jsons['RN']['features'][i]['properties']['name'] = 'Serra Caiada'
        
for i in range(len(dict_of_jsons['PB']['features'])):
    if(dict_of_jsons['PB']['features'][i]['properties']['name'] == 'Santarém'):
        dict_of_jsons['PB']['features'][i]['properties']['name'] = 'Joca Claudino'
    if(dict_of_jsons['PB']['features'][i]['properties']['name'] == 'Campo de Santana'):
        dict_of_jsons['PB']['features'][i]['properties']['name'] = 'Tacima'
    if(dict_of_jsons['PB']['features'][i]['properties']['name'] == 'Seridó'):
        dict_of_jsons['PB']['features'][i]['properties']['name'] = 'São Vicente do Seridó'

for i in range(len(dict_of_jsons['PE']['features'])):
    if(dict_of_jsons['PE']['features'][i]['properties']['name'] == 'Belém de São Francisco'):
        dict_of_jsons['PE']['features'][i]['properties']['name'] = 'Belém do São Francisco'
    if(dict_of_jsons['PE']['features'][i]['properties']['name'] == 'Iguaraci'):
        dict_of_jsons['PE']['features'][i]['properties']['name'] = 'Iguaracy'
    if(dict_of_jsons['PE']['features'][i]['properties']['name'] == 'Lagoa do Itaenga'):
        dict_of_jsons['PE']['features'][i]['properties']['name'] = 'Lagoa de Itaenga'
        
for i in range(len(dict_of_jsons['MG']['features'])):
    if(dict_of_jsons['MG']['features'][i]['properties']['name'] == 'Brasópolis'):
        dict_of_jsons['MG']['features'][i]['properties']['name'] = 'Brazópolis'
        
for i in range(len(dict_of_jsons['SP']['features'])):
    if(dict_of_jsons['SP']['features'][i]['properties']['name'] == 'Embu'):
        dict_of_jsons['SP']['features'][i]['properties']['name'] = 'Embu das Artes'
    if(dict_of_jsons['SP']['features'][i]['properties']['name'] == 'Moji Mirim'):
        dict_of_jsons['SP']['features'][i]['properties']['name'] = 'Mogi Mirim'
        
for i in range(len(dict_of_jsons['RS']['features'])):
    if(dict_of_jsons['RS']['features'][i]['properties']['name'] == 'Sant\' Ana do Livramento'):
        dict_of_jsons['RS']['features'][i]['properties']['name'] = 'Sant\'Ana do Livramento'

In [14]:
estados_nao_completos = []
municipios_n_encontrados = []
# Confirma que o geojson contém a mesma quantidade de municípios do banco de dados utilizado
for uf in states:
    key_name = str(uf)
    check = len(dict_of_jsons[key_name]["features"]) == len(dict_of_dff[key_name])
    if check == False:
        tam_json = len(dict_of_jsons[key_name]["features"])
        tam_df = len(dict_of_dff[key_name])
        estados_nao_completos.append(uf)
        print("{}: JSON - {} e DF - {}".format(uf, tam_json, tam_df))
lista = []
for j in estados_nao_completos:
    k = str(j)
    for i in range(len(dict_of_jsons[k]['features'])):
        lista.append(dict_of_jsons[k]['features'][i]['properties']['name'])
    lissst = list(dict_of_dff[k]['Município'])
    print('\nMUNICIPIOS FALTANTES DE {}\n'.format(j))
    for i in lissst:
        if(i not in lista):
            print("{}".format(i))
            municipios_n_encontrados.append(i)
    lista = []

PA: JSON - 143 e DF - 144
PI: JSON - 223 e DF - 224
SC: JSON - 293 e DF - 295
RS: JSON - 496 e DF - 497
MS: JSON - 78 e DF - 79

MUNICIPIOS FALTANTES DE PA

Mojuí dos Campos

MUNICIPIOS FALTANTES DE PI

Nazária

MUNICIPIOS FALTANTES DE SC

Balneário Rincão
Pescaria Brava

MUNICIPIOS FALTANTES DE RS

Pinto Bandeira

MUNICIPIOS FALTANTES DE MS

Paraíso das Águas


In [15]:
municipios_n_encontrados

['Mojuí dos Campos',
 'Nazária',
 'Balneário Rincão',
 'Pescaria Brava',
 'Pinto Bandeira',
 'Paraíso das Águas']

Não conseguimos plotar os dados dos **6** municípios listads acima, pois não encontramos o arquivo .json contendo seus respectivos contornos. Pesquisando no [Wikipédia](https://pt.wikipedia.org/wiki/Balne%C3%A1rio_Rinc%C3%A3o), descobri que esses municípios foram emancipados depois do json ser disponibilizado.

In [16]:
# Cria a barra de seleção dos dados a serem exibidos
drop_down = ipywidgets.Dropdown(options=densidades,
                                value = 'Hospitais a cada 100k habitantes',
                                disabled = False)

drop_down_estados = ipywidgets.Dropdown(options=states,
                                value = 'SP',
                                disabled = False)

hover = {densidades[0] : 'col_enfermeiros',
         densidades[1] : 'col_medicos',
         densidades[2] : 'col_hospitais',
         densidades[3] : 'col_leitos_int',
         densidades[4] : 'col_leitos_UTI',
         densidades[5] : 'col_leitos_UTI_COVID'}

Com os contornos e os dados já processados, fizemos a plotagem dos gráficos com dois layouts diferentes: <b>fundo claro</b> e <b>escuro</b>. A ideia é disponibilizar, futuramente, um botão para o próprio usuário escolher o layout preferido.

In [17]:
# Função que plota o gráfico
def generate_map(dado, estado):
    fig = go.Figure(data=go.Choropleth(
        geojson=dict_of_jsons["{}".format(estado)], # Arquivo json com os contornos dos municípios brasileiros
        z=dict_of_dff["{}".format(estado)]["{}".format(dado)], # Variavel que determina as cores do gráfico
        locations=dict_of_dff["{}".format(estado)]["Município"],
        featureidkey="properties.name", # Key do json com o nome dos municípios brasileiros
        colorscale="Teal", # Escala de cores a ser utilizada
        marker = dict(line = dict( color = "#212F3C", width = 1.1), opacity = 1),
        text = dict_of_dff["{}".format(estado)][hover["{}".format(dado)]],
        hoverinfo = "text",
        colorbar=dict(title = dict(text = "{}".format(dado),
                                 font = dict(family='tahoma', size = 16, color = "#D6DBDF"),
                                 side = "right"),
                      xanchor = 'left',
                      ypad = 30,
                      xpad = 20,
                      tickfont = dict(family = "tahoma", size = 12, color = "#D6DBDF"),
                      bordercolor = 'black',
                      bgcolor = "#212F3C",
                      len = 0.67
                     )
        ))
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(
        # Título do gráfico
        geo = dict(bgcolor="#212F3C"),
        paper_bgcolor = "#212F3C",
        title_font_family="tahoma",
        title_font_size = 20,
        hoverlabel=dict(
            bgcolor="#BFC9CA", 
            font_size=16,
            font = dict(color="#212F3C"),
            font_family="tahoma"
            ),
        title_text='<b>Dados de Leitos, Hospitais, Médicos e Enfermeiros por<br>Municípios brasileiros',
        title = dict(y = 0.9,
                     x = 0.5,
                     xanchor = 'center',
                     yanchor = 'top',
                     font = dict(color = "#D6DBDF" )),
        margin=dict(l=0, r=5, t=40, b=0)) # Margens da figura
    
    return fig

# Comando que faz a interação entre o dado selecionado em 'drop_down' e a plotagem do gráfico
ipywidgets.interact(generate_map, estado = drop_down_estados, dado = drop_down)

interactive(children=(Dropdown(description='dado', index=2, options=('Enfermeiros a cada 10k habitantes', 'Méd…

<function __main__.generate_map(dado, estado)>

In [18]:
# Função que plota o gráfico
def generate_map(dado, estado):
    fig = go.Figure(data=go.Choropleth(
        geojson=dict_of_jsons["{}".format(estado)], # Arquivo json com os contornos dos municípios brasileiros
        z=dict_of_dff["{}".format(estado)]["{}".format(dado)], # Variavel que determina as cores do gráfico
        locations=dict_of_dff["{}".format(estado)]["Município"],
        featureidkey="properties.name", # Key do json com o nome dos municípios brasileiros
        colorscale="Teal", # Escala de cores a ser utilizada
        marker = dict(line = dict( color = "#212F3C", width = 1.1), opacity = 1),
        text = dict_of_dff["{}".format(estado)][hover["{}".format(dado)]],
        hoverinfo = "text",
        colorbar=dict(title = dict(text = "{}".format(dado),
                                 font = dict(family='tahoma', size = 16, color = "#212F3C"),
                                 side = "right"),
                      xanchor = 'left',
                      ypad = 30,
                      xpad = 20,
                      tickfont = dict(family = "tahoma", size = 12, color = "#212F3C"),
                      bordercolor = 'black',
                      bgcolor = "#F4F6F6",
                      len = 0.67
                     )
        ))
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(
        # Título do gráfico
        geo = dict(bgcolor="#F4F6F6"),
        paper_bgcolor = "#F4F6F6",
        title_font_family="tahoma",
        title_font_size = 20,
        hoverlabel=dict(
            bgcolor='#34495E', 
            font_size=16,
            font = dict(color="#F4F6F6"),
            font_family="tahoma"
            ),
        title_text='<b>Dados de Leitos, Hospitais, Médicos e Enfermeiros por<br>Municípios brasileiros',
        title = dict(y = 0.9,
                     x = 0.5,
                     xanchor = 'center',
                     yanchor = 'top',
                     font = dict(color = "#212F3C" )),
        margin=dict(l=0, r=5, t=40, b=0)) # Margens da figura
    
    return fig

# Comando que faz a interação entre o dado selecionado em 'drop_down' e a plotagem do gráfico
ipywidgets.interact(generate_map, estado = drop_down_estados, dado = drop_down)

interactive(children=(Dropdown(description='dado', index=2, options=('Enfermeiros a cada 10k habitantes', 'Méd…

<function __main__.generate_map(dado, estado)>

Com esse notebook, conseguimos completar o nosso objetivo original:
    
- [X] População dos municípios brasileiros
- [X] Quantidade de hospitais públicos e não públicos por município brasileiro
- [X] Quantidade de enfermeiros do SUS ou não por município brasileiro
- [X] Quantidade de médicos do SUS ou não por município brasileiro
- [X] Quantidade de leitos de internação do SUS ou não por município brasileiro
- [X] Quantidade de leitos de UTI do SUS ou não por município brasileiro
- [X] Quantidade de leitos de UTI para COVID-19 do SUS ou não por município brasileiro
- [X] Enfermeiros a cada 10k habitantes por município brasileiro
- [X] Médicos a cada 10k habitantes por município brasileiro
- [X] Hospitais a cada 10k habitantes por município brasileiro
- [X] Leitos (internação) a cada 10k habitantes por município brasileiro
- [X] Leitos UTI a cada 10k habitantes por município brasileiro
- [X] Leitos UTI COVID-19 a cada 10k habitantes por município brasileiro